<a href="https://colab.research.google.com/github/mmuokubamichael/12th-place-solution-Turtle-Recall-Conservation-Challenge/blob/main/tutle_classification_solution_with_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

SOURCE_URL = 'https://storage.googleapis.com/dm-turtle-recall/images.tar'
IMAGE_DIR = './turtle_recall/images'
TAR_PATH = os.path.join(IMAGE_DIR, os.path.basename(SOURCE_URL))
EXPECTED_IMAGE_COUNT = 13891

%sx mkdir --parents "{IMAGE_DIR}"
if len(os.listdir(IMAGE_DIR)) != EXPECTED_IMAGE_COUNT:
  %sx wget --no-check-certificate -O "{TAR_PATH}" "{SOURCE_URL}"
  %sx tar --extract --file="{TAR_PATH}" --directory="{IMAGE_DIR}"
  %sx rm "{TAR_PATH}"

print(f'The total number of images is: {len(os.listdir(IMAGE_DIR))}')

In [ ]:
import pandas as pd 
import requests
import io
import urllib.parse

BASE_URL = 'https://storage.googleapis.com/dm-turtle-recall/'

def read_csv_from_web(file_name):
  url = urllib.parse.urljoin(BASE_URL, file_name)  
  content = requests.get(url).content
  return pd.read_csv(io.StringIO(content.decode('utf-8')))

# Read in csv files.
train = read_csv_from_web('train.csv')
test = read_csv_from_web('test.csv')
sample_submission = read_csv_from_web('sample_submission.csv')

# Convert image_location strings to lowercase.
for df in [train, test]:
  df.image_location = df.image_location.apply(lambda x: x.lower())
  assert set(df.image_location.unique()) == set(['left', 'right', 'top'])

In [ ]:
train.shape

In [ ]:
import numpy as np


In [ ]:
turtle_ids = sorted(np.unique(train.turtle_id)) + ['new_turtle']
labels = dict(zip(turtle_ids, np.arange(len(turtle_ids))))
label_lookup = {v: k for k, v in labels.items()}
num_classes = len(labels)
image_to_turtle = dict(zip(train.image_id, train.turtle_id))

image_files = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) 
              if f.split('.')[0] in train.image_id.values]

image_ids = [os.path.basename(f).split('.')[0] for f in image_files]
image_turtle_ids = [image_to_turtle[id] for id in image_ids]

In [ ]:
y_target= pd.DataFrame(image_turtle_ids,columns=["target"])

In [ ]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
target_encode=LabelEncoder()
y_target['target']=target_encode.fit_transform(y_target['target'])

In [ ]:
target_onehot=OneHotEncoder()
main_target=target_onehot.fit_transform(y_target[['target']]).toarray()

In [ ]:
main_target

In [ ]:
import functools
from PIL import Image  # Image utilities.
import tqdm

In [ ]:
def crop_and_resize(pil_img):
  """Crops square from center of image and resizes to (224, 224)."""
  w, h = pil_img.size
  crop_size = min(w, h)
  crop = pil_img.crop(
      ((w - crop_size) // 2,
       (h - crop_size) // 2,
       (w + crop_size) // 2,
       (h + crop_size) // 2))
  return crop.resize((224, 224))


tqdm.tqdm._instances.clear()
loaded_images = [crop_and_resize(Image.open(f)) for f in tqdm.tqdm(image_files)]

In [ ]:
train=[]
for x in loaded_images:
  train.append(np.array(x))


In [ ]:
x_train=np.array(train)
y_train=np.array(main_target)

In [ ]:
x_train.shape,y_train.shape


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.resnet import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras import Sequential
from keras.layers import Dense, LSTM
from keras.layers import TimeDistributed

In [ ]:
def estimator(model):
  for layer in model.layers:
    layer.trainable = False
  # add new classifier layers
  flat1 = Flatten()(model.layers[-1].output)
  class1 = Dense(1024, activation='relu')(flat1)
  output = Dense(100, activation='softmax')(class1)
  # define new model
  model = Model(inputs=model.inputs, outputs=output)
  model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=[
          'categorical_accuracy'
      ])
  return model

In [ ]:
resnet=ResNet50(include_top=False, input_shape=(224, 224, 3))
nasnet=InceptionResNetV2(include_top=False, input_shape=(224, 224, 3))
vg16 = VGG16(include_top=False, input_shape=(224, 224, 3))
inception=InceptionV3(include_top=False, input_shape=(224, 224, 3))
xception=Xception(include_top=False, input_shape=(224, 224, 3))
resnet101v2=ResNet101V2(include_top=False, input_shape=(224, 224, 3))
resnet152v2=ResNet152V2(include_top=False, input_shape=(224, 224, 3))


vg16=estimator(vg16)
inception=estimator(inception)
xception=estimator(xception)
resnet101v2=estimator(resnet101v2)
resnet152v2=estimator(resnet152v2)
resnet=estimator(resnet)
nasnet=estimator(nasnet)

In [ ]:
nasnet.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 52s 421ms/step - loss: 7638.5840 - categorical_accuracy: 0.0117
Epoch 2/30
68/68 [==============================] - 28s 411ms/step - loss: 16.2496 - categorical_accuracy: 0.0131
Epoch 3/30
68/68 [==============================] - 28s 409ms/step - loss: 4.5990 - categorical_accuracy: 0.0135
Epoch 4/30
68/68 [==============================] - 28s 409ms/step - loss: 4.5937 - categorical_accuracy: 0.0317
Epoch 5/30
68/68 [==============================] - 28s 408ms/step - loss: 4.5880 - categorical_accuracy: 0.0359
Epoch 6/30
68/68 [==============================] - 28s 409ms/step - loss: 4.5823 - categorical_accuracy: 0.0359
Epoch 7/30
68/68 [==============================] - 28s 409ms/step - loss: 4.5765 - categorical_accuracy: 0.0359
Epoch 8/30
68/68 [==============================] - 28s 409ms/step - loss: 4.5706 - categorical_accuracy: 0.0359
Epoch 9/30
68/68 [==============================] - 28s 408ms/step - loss: 4.5649 - categori

In [ ]:
resnet152v2.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 48s 535ms/step - loss: 15332.4570 - categorical_accuracy: 0.0135
Epoch 2/30
68/68 [==============================] - 36s 529ms/step - loss: 6.2713 - categorical_accuracy: 0.0089
Epoch 3/30
68/68 [==============================] - 36s 527ms/step - loss: 4.5955 - categorical_accuracy: 0.0205
Epoch 4/30
68/68 [==============================] - 36s 529ms/step - loss: 4.5900 - categorical_accuracy: 0.0364
Epoch 5/30
68/68 [==============================] - 36s 529ms/step - loss: 4.5839 - categorical_accuracy: 0.0364
Epoch 6/30
68/68 [==============================] - 36s 529ms/step - loss: 4.5776 - categorical_accuracy: 0.0364
Epoch 7/30
68/68 [==============================] - 36s 527ms/step - loss: 4.5712 - categorical_accuracy: 0.0364
Epoch 8/30
68/68 [==============================] - 36s 529ms/step - loss: 4.5647 - categorical_accuracy: 0.0364
Epoch 9/30
68/68 [==============================] - 36s 529ms/step - loss: 4.5616 - categori

In [ ]:
resnet101v2.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 33s 373ms/step - loss: 9959.6768 - categorical_accuracy: 0.0126
Epoch 2/30
68/68 [==============================] - 25s 374ms/step - loss: 11.6194 - categorical_accuracy: 0.0163
Epoch 3/30
68/68 [==============================] - 25s 373ms/step - loss: 4.6594 - categorical_accuracy: 0.0172
Epoch 4/30
68/68 [==============================] - 25s 373ms/step - loss: 4.6023 - categorical_accuracy: 0.0294
Epoch 5/30
68/68 [==============================] - 25s 373ms/step - loss: 4.5846 - categorical_accuracy: 0.0364
Epoch 6/30
68/68 [==============================] - 25s 374ms/step - loss: 4.5827 - categorical_accuracy: 0.0364
Epoch 7/30
68/68 [==============================] - 25s 373ms/step - loss: 4.5727 - categorical_accuracy: 0.0364
Epoch 8/30
68/68 [==============================] - 25s 374ms/step - loss: 4.5650 - categorical_accuracy: 0.0368
Epoch 9/30
68/68 [==============================] - 25s 373ms/step - loss: 4.5590 - categori

In [ ]:
xception.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 29s 356ms/step - loss: 148.4766 - categorical_accuracy: 0.0191
Epoch 2/30
68/68 [==============================] - 24s 350ms/step - loss: 5.0676 - categorical_accuracy: 0.0434
Epoch 3/30
68/68 [==============================] - 24s 350ms/step - loss: 4.6108 - categorical_accuracy: 0.0494
Epoch 4/30
68/68 [==============================] - 24s 349ms/step - loss: 4.5056 - categorical_accuracy: 0.0559
Epoch 5/30
68/68 [==============================] - 24s 350ms/step - loss: 4.4352 - categorical_accuracy: 0.0620
Epoch 6/30
68/68 [==============================] - 24s 350ms/step - loss: 4.4223 - categorical_accuracy: 0.0611
Epoch 7/30
68/68 [==============================] - 24s 350ms/step - loss: 4.4547 - categorical_accuracy: 0.0676
Epoch 8/30
68/68 [==============================] - 24s 350ms/step - loss: 4.3249 - categorical_accuracy: 0.0695
Epoch 9/30
68/68 [==============================] - 24s 350ms/step - loss: 4.3289 - categorica

In [ ]:
vg16.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 29s 261ms/step - loss: 35.0159 - categorical_accuracy: 0.0233
Epoch 2/30
68/68 [==============================] - 17s 250ms/step - loss: 4.0513 - categorical_accuracy: 0.1245
Epoch 3/30
68/68 [==============================] - 17s 250ms/step - loss: 3.4847 - categorical_accuracy: 0.2247
Epoch 4/30
68/68 [==============================] - 17s 250ms/step - loss: 2.8769 - categorical_accuracy: 0.3273
Epoch 5/30
68/68 [==============================] - 17s 251ms/step - loss: 2.3154 - categorical_accuracy: 0.4368
Epoch 6/30
68/68 [==============================] - 17s 251ms/step - loss: 1.8952 - categorical_accuracy: 0.5096
Epoch 7/30
68/68 [==============================] - 17s 250ms/step - loss: 1.3272 - categorical_accuracy: 0.6578
Epoch 8/30
68/68 [==============================] - 17s 250ms/step - loss: 0.9147 - categorical_accuracy: 0.7450
Epoch 9/30
68/68 [==============================] - 17s 250ms/step - loss: 1.0350 - categorical

In [ ]:
inception.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 19s 190ms/step - loss: 454.4582 - categorical_accuracy: 0.0159
Epoch 2/30
68/68 [==============================] - 12s 183ms/step - loss: 4.6528 - categorical_accuracy: 0.0121
Epoch 3/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5982 - categorical_accuracy: 0.0252
Epoch 4/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5799 - categorical_accuracy: 0.0275
Epoch 5/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5690 - categorical_accuracy: 0.0373
Epoch 6/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5646 - categorical_accuracy: 0.0373
Epoch 7/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5584 - categorical_accuracy: 0.0373
Epoch 8/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5532 - categorical_accuracy: 0.0368
Epoch 9/30
68/68 [==============================] - 12s 183ms/step - loss: 4.5464 - categorica

In [ ]:
resnet.fit(x_train,y_train,epochs=30)

Epoch 1/30
68/68 [==============================] - 21s 243ms/step - loss: 32.0476 - categorical_accuracy: 0.0238
Epoch 2/30
68/68 [==============================] - 16s 241ms/step - loss: 4.3691 - categorical_accuracy: 0.0583
Epoch 3/30
68/68 [==============================] - 16s 242ms/step - loss: 3.9712 - categorical_accuracy: 0.1282
Epoch 4/30
68/68 [==============================] - 16s 243ms/step - loss: 3.3959 - categorical_accuracy: 0.2098
Epoch 5/30
68/68 [==============================] - 17s 243ms/step - loss: 2.9290 - categorical_accuracy: 0.2984
Epoch 6/30
68/68 [==============================] - 16s 242ms/step - loss: 2.5740 - categorical_accuracy: 0.3534
Epoch 7/30
68/68 [==============================] - 16s 243ms/step - loss: 1.9101 - categorical_accuracy: 0.4890
Epoch 8/30
68/68 [==============================] - 17s 244ms/step - loss: 1.3372 - categorical_accuracy: 0.6159
Epoch 9/30
68/68 [==============================] - 17s 244ms/step - loss: 0.8361 - categorical

In [ ]:
from sklearn.model_selection import KFold
skf=KFold(n_splits=10)

In [ ]:
def model_train(estimator,train,label,test):
    test_pred_y = np.zeros((test.shape[0],100))
    for count, (train_index,test_index) in enumerate(skf.split(train,label)):
        x_train,x_test = train[train_index],train[test_index]
        y_train,y_test = label[train_index],label[test_index]
        print(f'========================Fold{count +1}==========================')
        estimator.fit(x_train,y_train,epochs=30)
                              
        
        test_pred= estimator.predict(test)
        test_pred_y +=test_pred
        
    return test_pred_y

In [ ]:
predicts=model_train(resnet,x_train,y_train,test_set)

========================Fold1==========================
Epoch 1/30
61/61 [==============================] - 15s 244ms/step - loss: 9.4204e-04 - categorical_accuracy: 1.0000
Epoch 2/30
61/61 [==============================] - 15s 242ms/step - loss: 8.8781e-04 - categorical_accuracy: 1.0000
Epoch 3/30
61/61 [==============================] - 15s 243ms/step - loss: 8.3726e-04 - categorical_accuracy: 1.0000
Epoch 4/30
61/61 [==============================] - 15s 242ms/step - loss: 7.9485e-04 - categorical_accuracy: 1.0000
Epoch 5/30
61/61 [==============================] - 15s 242ms/step - loss: 7.5415e-04 - categorical_accuracy: 1.0000
Epoch 6/30
61/61 [==============================] - 15s 242ms/step - loss: 7.1603e-04 - categorical_accuracy: 1.0000
Epoch 7/30
61/61 [==============================] - 15s 242ms/step - loss: 6.8559e-04 - categorical_accuracy: 1.0000
Epoch 8/30
61/61 [==============================] - 15s 242ms/step - loss: 6.4932e-04 - categorical_accuracy: 1.0000
Epoch 9/

In [ ]:
predicts.shape

(490, 100)

In [ ]:
tqdm.tqdm._instances.clear()
test_image_files = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) 
                    if f.split('.')[0] in test.image_id.values]
test_image_ids = [os.path.basename(f).split('.')[0] for f in test_image_files]
loaded_test_images = [crop_and_resize(Image.open(f)) for f in tqdm.tqdm(test_image_files)]

100%|██████████| 490/490 [00:20<00:00, 23.92it/s]


In [ ]:
test=[]
for x in loaded_test_images:
  test.append(np.array(x))


In [ ]:
test_set=np.array(test)

In [ ]:
prediction_resnet=resnet.predict(test_set)
prediction_vg16=vg16.predict(test_set)

In [ ]:
prediction=prediction_resnet + prediction_vg16

In [ ]:
prediction.shape

(490, 100)

In [ ]:
prediction_vg16=vg16.predict(test_set)
prediction_inception=inception.predict(test_set)
prediction_xception=xception.predict(test_set)
prediction_resnet101v2=resnet101v2.predict(test_set)
prediction_resnet152v2=resnet152v2.predict(test_set)
prediction_nasnet=nasnet.predict(test_set)

(100,)

In [ ]:
prediction=predicts

In [ ]:
classes=[]
for i in range(490):
  class_prob=prediction[i]
  top_values_index = sorted(range(len(class_prob)), key=lambda i: class_prob[i])[-5:]
  top_values_index.reverse()
  classes.append(top_values_index)


In [ ]:
classes

In [ ]:
target_encode

In [ ]:
transform_full=[]
for x in classes:
  pred=target_encode.inverse_transform(x)
  transform_full.append(pred)
  

In [ ]:
t_id_C0wevyOl', 't_id_EEbWq5Pj', 't_id_G5eoqwD8', 't_id_8b8sprYe

In [ ]:
transform_full

In [ ]:
transform_full

In [ ]:
transform_full

In [ ]:
sample_submission.head()

,image_id,prediction1,prediction2,prediction3,prediction4,prediction5
0,ID_6NEDKOYZ,new_turtle,t_id_d6aYXtor,t_id_qZ0iZYsC,new_turtle,t_id_d6aYXtor
1,ID_57QZ4S9N,new_turtle,t_id_d6aYXtor,t_id_qZ0iZYsC,new_turtle,t_id_d6aYXtor
2,ID_OCGGJS5X,new_turtle,t_id_d6aYXtor,t_id_qZ0iZYsC,new_turtle,t_id_d6aYXtor
3,ID_R2993S3S,new_turtle,t_id_d6aYXtor,t_id_qZ0iZYsC,new_turtle,t_id_d6aYXtor
4,ID_2E011NB0,new_turtle,t_id_d6aYXtor,t_id_qZ0iZYsC,new_turtle,t_id_d6aYXtor


In [ ]:
test_image_ids

In [ ]:
transform_full_data_frame=pd.DataFrame(transform_full,columns=['prediction1','prediction2','prediction3','prediction4','prediction5'])

In [ ]:
transform_full_data_frame["image_id"]=test_image_ids

In [ ]:
transform_full_data_frame.head()

,prediction1,prediction2,prediction3,prediction4,prediction5,image_id
0,t_id_EEbWq5Pj,t_id_X3bKBWYW,t_id_2QmcRkNj,t_id_8b8sprYe,t_id_G5eoqwD8,ID_JUQM4S99
1,t_id_8b8sprYe,t_id_n2FBHk6d,t_id_9GFmcOd5,t_id_SwQZGIpa,t_id_D0gA44av,ID_DFT8JWF0
2,t_id_VP2NW7aV,t_id_2E8o5Jtl,t_id_ROFhVsy2,t_id_HxxqrdTx,t_id_AOWArhGb,ID_3QP1M0SV
3,t_id_Ts5LyVQz,t_id_AOWArhGb,t_id_SwQZGIpa,t_id_C0wevyOl,t_id_fxTQ5vHC,ID_G5S2G6IF
4,t_id_4ZfTUmwL,t_id_Ts5LyVQz,t_id_D0gA44av,t_id_B7LaSiac,t_id_d6aYXtor,ID_91E1O0FJ


In [ ]:
transform_full_data_frame=transform_full_data_frame[["image_id","prediction1","prediction2","prediction3","prediction4","prediction5"]]

In [ ]:
transform_full_data_frame.to_csv("firstsub2.csv",index=False)

In [ ]:
class_prob = [0.98,0.50,0.60,0.90,0.87,0.79,0.87]
top_values_index = sorted(range(len(class_prob)), key=lambda i: class_prob[i])[-5:]

In [ ]:
top_values_index.reverse()

In [ ]:
top_values_index

[0, 3, 6, 4, 5]